<a href="https://colab.research.google.com/github/kevinMGII/Grupo7-Practica1/blob/main/Grupo7_Practica1_ModeloFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MODELO FINAL**. Predicción del Abandono de Empleados.

Grupo 7. Miembros:

*   *Kevin Medina García, 100495893.*
*   *Bárbara Sánchez Moratalla, 100495857.*

---


En este cuaderno entrenamos el modelo final y lo usamos para hacer predicciones en los datos de competición. Los resultados se guardan en el archivo predicciones.csv.

- Carga de librerías necesarias

In [34]:
import pandas as pd
import joblib
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.metrics import balanced_accuracy_score

- Lectura de datos de competición y modelo final

In [35]:
data_test = pd.read_csv("./attrition_competition_12.csv.gz")                    # Cargamos el conjunto de datos de test utilizando el almacenamiento local de Google Colab.
modelo_final = joblib.load("./modelo_final.pkl")                                # Cargamos el modelo final

- Realizamos las predicciones

In [36]:
# 1. DEFINICIÓN DE VARIABLES

categorical_features = ['Department', 'EducationField', 'JobRole']              # Lista de atributos categóricos
ordinal_features = ['BusinessTravel', 'Gender', 'MaritalStatus']                # Lista de atributos ordinale
drop_columns = ["EmployeeID", "EmployeeCount", "Over18", "StandardHours"]       # Lista con las columnas innecesarias. Análisis anteriores.

ordinal_mapping = [                                                             # Lista con los posibles valores de las variables ordinales
    ['Non-Travel', 'Travel_Rarely', 'Travel_Frequently'],                       # Posibles valores de BusinnesTravel
    ['Male', 'Female'],                                                         # Posibles valores de Gender
    ['Married', 'Divorced', 'Single']                                           # Posibles valores de MaritalStatus
]

print("Completado - Definición de Variables.")                                  # Mostramos un mensaje por pantalla, funcionamiento correcto

Completado - Definición de Variables.


In [37]:
# 2.- PREPROCESAMIENTO DE DATOS

preprocessor = ColumnTransformer(                                               # Utilizamos ColumnTransformer para aplicar transformar columnas no numéricas a numéricas
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),             # Aplicamos OneHotEncoding a las variables categóricas
        ('ord', OrdinalEncoder(categories=ordinal_mapping), ordinal_features)   # Aplicamos OrdinalEndocing a las variables ordinales
    ],
    remainder='passthrough'                                                     # Mantenemos las columnas numéricas sin cambios
)

data_test = data_test.drop(columns=drop_columns)                                            # Eliminamos las columnas innecesarias
data_test = data_test.drop(columns=["Attrition"], errors='ignore')                          # Eliminamos la columna Attrition del conjunto de test

data_test = preprocessor.fit_transform(data_test)                                           # Eliminamos las columnas innecesarias
data_test = pd.DataFrame(data_test, columns=preprocessor.get_feature_names_out())           # Utilizamos el preprocessor.fit para aplicarlo

print("Completado - Preprocesamiento de Datos.")                                # Mostramos un mensaje por pantalla, funcionamiento correcto

Completado - Preprocesamiento de Datos.


In [38]:
# 3. PREDICCIÓN Y RESULTADOS

y_pred_test = modelo_final.predict(data_test)                                   # Generamos predicciones con el modelo entrenado, predicciones sobre los datos de test
df_predicciones = pd.DataFrame(y_pred_test, columns=["Predicción"])             # Creamos un dataframe con las predicciones

print(df_predicciones["Predicción"].value_counts())                             # Mostramos un mensaje por pantalla, predicciones realizadas

Predicción
0    1237
1     233
Name: count, dtype: int64
